## Load the data

In [1]:
import src.utils as utils

data = utils.load_data('data/training.csv')

In [2]:
data.shape

(12906, 83)

## Preprocessing

In [3]:
import src.data_preprocessing as prep

data_no_cat = prep.drop_cat(data)
X_train, X_test, y_train, y_test = prep.split_data(data_no_cat)

## Training

In [10]:
import xgboost as xgb

xgb_classifier = xgb.XGBClassifier(
    objective='binary:logistic', 
    max_depth=15,
    learning_rate=0.01,
    n_estimators=500
)

xgb_classifier.fit(X_train, y_train)
y_pred = xgb_classifier.predict(X_test)

## Evaluation

In [11]:
import src.evaluation as eval

res = eval.evaluate_model(y_test,y_pred)
res

{'Accuracy': 0.5968241673121611,
 'Precision': 0.6458966565349544,
 'Recall': 0.7884972170686456,
 'F1 Score': 0.7101086048454469,
 'ROC AUC': 0.532072442731214}

## Create Kaggle submision

In [ ]:
# TO DO


## Save results

In [ ]:
#TO DO